In [0]:
#1.Ingest sample order data into a Spark DataFrame.
#used kaggle and found the dataset then modified as per requirement
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
#/Volumes/chubb/default/deltalake/ShopEZ_orders.csv
spark = SparkSession.builder.appName("ShopEZ").getOrCreate()
# Loading shopesz dataset
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/Volumes/chubb/default/deltalake/ShopEZ_orders.csv")
df.show(5)
df.printSchema()

+---------+-------------------+-----------+-------+--------+-------+-------+
| order_id|    order_timestamp|customer_id|country|currency| amount| status|
+---------+-------------------+-----------+-------+--------+-------+-------+
|ORD100000|2023-05-12 14:23:45|   CUST1863|     CA|     CAD|4937.69|CREATED|
|ORD100001|2023-11-26 01:45:42|   CUST7805|     UK|     GBP|3138.91|CREATED|
|ORD100002|2024-04-23 03:53:06|   CUST5839|     AU|     AUD|1018.75|   PAID|
|ORD100003|2023-12-10 05:00:38|   CUST4763|     CA|     CAD|1395.14|   PAID|
|ORD100004|2023-06-05 02:15:41|   CUST5409|     IN|     INR|1174.63|CREATED|
+---------+-------------------+-----------+-------+--------+-------+-------+
only showing top 5 rows
root
 |-- order_id: string (nullable = true)
 |-- order_timestamp: timestamp (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- country: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- status: string (nullabl

In [0]:
#2.Adding a derived column called order_date using order_timestamp
#extracts year-month-day 
df=df.withColumn("order_date", to_date(col("order_timestamp")))
df.show(5)
df.printSchema()

+---------+-------------------+-----------+-------+--------+-------+-------+----------+
| order_id|    order_timestamp|customer_id|country|currency| amount| status|order_date|
+---------+-------------------+-----------+-------+--------+-------+-------+----------+
|ORD100000|2023-05-12 14:23:45|   CUST1863|     CA|     CAD|4937.69|CREATED|2023-05-12|
|ORD100001|2023-11-26 01:45:42|   CUST7805|     UK|     GBP|3138.91|CREATED|2023-11-26|
|ORD100002|2024-04-23 03:53:06|   CUST5839|     AU|     AUD|1018.75|   PAID|2024-04-23|
|ORD100003|2023-12-10 05:00:38|   CUST4763|     CA|     CAD|1395.14|   PAID|2023-12-10|
|ORD100004|2023-06-05 02:15:41|   CUST5409|     IN|     INR|1174.63|CREATED|2023-06-05|
+---------+-------------------+-----------+-------+--------+-------+-------+----------+
only showing top 5 rows
root
 |-- order_id: string (nullable = true)
 |-- order_timestamp: timestamp (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- country: string (nullable = true)
 |-- cu

In [0]:
#3.Write the DataFrame as a Delta table partitioned by country and order_date
# Write dataframe as delta table
# Partition by country and order_date
# Save as managed delta table
df.write.format("delta") \
    .mode("overwrite") \
    .partitionBy("country", "order_date") \
    .saveAsTable("chubb.default.orders")

In [0]:
#4.Verify the partition structure in the storage path.
#Unity Catalog managed tables ALWAYS show location as null
display(spark.sql("DESCRIBE DETAIL chubb.default.orders"))

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics,clusterByAuto
delta,b36fa122-9391-4443-b0ac-f2b51b255c67,chubb.default.orders,null,,2025-12-01T17:10:29.243Z,2025-12-01T17:11:31.000Z,"List(country, order_date)",List(),3455,6365722,Map(delta.enableDeletionVectors -> true),3,7,"List(appendOnly, deletionVectors, invariants)","Map(numRowsDeletedByDeletionVectors -> 0, numDeletionVectors -> 0)",false


In [0]:
%sql
-- Show table partitions as unity catalog is on
SHOW PARTITIONS chubb.default.orders;

country,order_date
UK,2024-01-30
AU,2023-05-31
UK,2023-07-05
UK,2024-07-25
UK,2023-03-18
CA,2024-10-12
UK,2023-05-22
CA,2024-04-25
UK,2023-10-10
US,2024-07-03


In [0]:
## Run sql to show partitions
spark.sql("SHOW PARTITIONS chubb.default.orders").show(truncate=False)
# If truncate is false then it displays full output without truncation


+-------+----------+
|country|order_date|
+-------+----------+
|UK     |2024-01-30|
|AU     |2023-05-31|
|UK     |2023-07-05|
|UK     |2024-07-25|
|UK     |2023-03-18|
|CA     |2024-10-12|
|UK     |2023-05-22|
|CA     |2024-04-25|
|UK     |2023-10-10|
|US     |2024-07-03|
|US     |2024-01-19|
|IN     |2024-01-06|
|CA     |2023-08-30|
|IN     |2024-11-27|
|US     |2024-05-21|
|UK     |2024-08-28|
|CA     |2024-08-19|
|AU     |2024-12-07|
|UK     |2023-05-10|
|AU     |2023-02-17|
+-------+----------+
only showing top 20 rows


In [0]:
#5.Run queries that demonstrate partition pruning
# Read delta table using table name
df_delta = spark.table("chubb.default.orders")
# 1.Filter by country to prune partitions
df_delta.filter(col("country") == "IN").show()
# 2.Filter by order_date to prune date partitions
df_delta.filter(col("order_date") == "2024-01-15").show()
# 3.Filter by both country and date to prune more partitions
df_delta.filter(
    (col("country") == "US") &
    (col("order_date") == "2024-01-20")
).show()
# 4.Count rows after pruning
df_delta.filter(col("country") == "UK") \
    .groupBy("country", "order_date") \
    .count() \
    .show()

+---------+-------------------+-----------+-------+--------+-------+---------+----------+
| order_id|    order_timestamp|customer_id|country|currency| amount|   status|order_date|
+---------+-------------------+-----------+-------+--------+-------+---------+----------+
|ORD106074|2023-03-27 03:41:37|   CUST2255|     IN|     INR|1053.78|CANCELLED|2023-03-27|
|ORD104604|2023-03-27 06:14:47|   CUST5982|     IN|     INR|1235.16|CANCELLED|2023-03-27|
|ORD103695|2023-03-27 23:35:44|   CUST2841|     IN|     INR| 572.71|CANCELLED|2023-03-27|
|ORD101791|2023-03-27 11:31:46|   CUST6104|     IN|     INR|1071.53|  CREATED|2023-03-27|
|ORD100987|2023-03-27 16:10:14|   CUST7716|     IN|     INR|4722.76|CANCELLED|2023-03-27|
|ORD109946|2023-05-18 12:01:20|   CUST8371|     IN|     INR|3490.06|CANCELLED|2023-05-18|
|ORD103629|2023-05-18 00:18:40|   CUST8453|     IN|     INR|3121.65|CANCELLED|2023-05-18|
|ORD102739|2023-05-18 11:47:01|   CUST5274|     IN|     INR|2884.98|     PAID|2023-05-18|
|ORD101595

In [0]:
spark.sql("DESCRIBE DETAIL chubb.default.orders").show(truncate=False)


+------+------------------------------------+--------------------+-----------+--------+-----------------------+-------------------+---------------------+-----------------+--------+-----------+-------------------------------------+----------------+----------------+-----------------------------------------+---------------------------------------------------------------+-------------+
|format|id                                  |name                |description|location|createdAt              |lastModified       |partitionColumns     |clusteringColumns|numFiles|sizeInBytes|properties                           |minReaderVersion|minWriterVersion|tableFeatures                            |statistics                                                     |clusterByAuto|
+------+------------------------------------+--------------------+-----------+--------+-----------------------+-------------------+---------------------+-----------------+--------+-----------+-------------------------------------+

In [0]:
# Importing the DeltaTable class to work with existing Delta tables
from delta.tables import DeltaTable
# Load the Delta table using the given name
delta= DeltaTable.forName(spark, "chubb.default.orders")
# Display the transaction history of the Delta table
# Each entry includes version number, operation type timestamp, user info, and other metadata. Useful for auditing and debugging.
delta.history().show(truncate=False)

+-------+-------------------+--------------+-----------------------+---------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+-----------------+------------------------+-----------+-----------------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------+------------+--------------------------------------------------+
|version|timestamp          |userId        |userName               |operation                        |operationParameters                                                                                                                                                          |job |notebook         |clusterId               |readVersion|isolationLevel   |isBlindAppend|operationMetrics                                     

In [0]:
spark.sql("""
SELECT *
FROM chubb.default.orders
WHERE country = 'IN'
""").explain(extended=True)

== Parsed Logical Plan ==
'Project [*]
+- 'Filter ('country = IN)
   +- 'UnresolvedRelation [chubb, default, orders], [], false

== Analyzed Logical Plan ==
order_id: string, order_timestamp: timestamp, customer_id: string, country: string, currency: string, amount: double, status: string, order_date: date, payment_method: string, coupon_code: string
Project [order_id#46479, order_timestamp#46480, customer_id#46481, country#46482, currency#46483, amount#46484, status#46485, order_date#46486, payment_method#46487, coupon_code#46488]
+- Filter (country#46482 = IN)
   +- SubqueryAlias chubb.default.orders
      +- Relation chubb.default.orders[order_id#46479,order_timestamp#46480,customer_id#46481,country#46482,currency#46483,amount#46484,status#46485,order_date#46486,payment_method#46487,coupon_code#46488] parquet

== Optimized Logical Plan ==
Filter (isnotnull(country#46482) AND (country#46482 = IN))
+- Relation chubb.default.orders[order_id#46479,order_timestamp#46480,customer_id#46481

In [0]:
%sql
ANALYZE TABLE chubb.default.orders COMPUTE STATISTICS FOR ALL COLUMNS;

In [0]:
# Task - 6 : Demonstrate Delta Lake Time Travel
from delta.tables import DeltaTable

# Load the Delta table using the table name.
# Unity Catalog managed tables use forName() instead of forPath().
dt = DeltaTable.forName(spark, "chubb.default.orders")
# Show the transaction history of the Delta table.
# Each entry contains version number, timestamp, read/write info,
# operation type (WRITE, UPDATE), and useful metadata.
dt.history().show(truncate=False)
# Update rows in the Delta table where status = 'CREATED'
# This will change those rows to 'PAID' and create a new version.
dt.update(
    condition="status = 'CREATED'",
    set={"status": "'PAID'"}
)
# Show history again after the update
dt.history().show(truncate=False)
# TIME TRAVEL: Read an older version of the table
# versionAsOf = 0 loads the very first version of the table
old_df = spark.read \
    .option("versionAsOf", 0) \
    .table("chubb.default.orders")
old_df.show(truncate=False)
# Read the current/latest version of the Delta table
current_df = spark.table("chubb.default.orders")

current_df.show(truncate=False)


+-------+-------------------+--------------+-----------------------+---------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
#7.Demonstrate Schema Evolution by adding new columns
df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_timestamp: timestamp (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- country: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- status: string (nullable = true)
 |-- order_date: date (nullable = true)



In [0]:
from pyspark.sql.functions import col, lit, when, rand
df_new = (
    df
    # Add payment_method column
    .withColumn(
        "payment_method",
        when(col("status") == "CREATED", "NOT COMPLETED")
        .when(
            col("status") == "CANCELLED",
            when(rand() < 0.5, "REFUNDED").otherwise(lit(None))
        )
        .when(
            col("status") == "PAID",
            when(col("amount") < 1000,
                 when(rand() < 0.6, "COD").otherwise("UPI"))
            .when(col("amount") < 4000,
                  when(rand() < 0.3, "COD")
                  .when(rand() < 0.5, "UPI")
                  .otherwise("CARD"))
            .otherwise(
                when(rand() < 0.4, "UPI").otherwise("CARD")
            )
        )
        .otherwise(lit(None))
    )
    # Add coupon_code column
    .withColumn(
        "coupon_code",
        when(col("amount") >= 3000, "SAVE300")
        .when(col("amount") >= 1500, "SAVE150")
        .when(col("amount") >= 500, "FLAT50")
        .otherwise(lit(None))
    )
)
df_new.printSchema()


root
 |-- order_id: string (nullable = true)
 |-- order_timestamp: timestamp (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- country: string (nullable = true)
 |-- currency: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- status: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- coupon_code: string (nullable = true)



In [0]:
#Writing new data to the Delta table with schema evolution
df_new.write \
    .format("delta") \
    .mode("append") \
    .option("mergeSchema", "true") \
    .saveAsTable("chubb.default.orders")

In [0]:
#8.1 Demonstrate Delta Lake UPDATE.
#Cancel orders where amount is between 50 and 200
updated_rows = delta.update(
    condition="amount >= 50 AND amount <= 200",   
    set={"status": "'CANCELLED'"}
)

In [0]:
#8.2 Demonstrate Delta Lake DELETE operation.
#Deleting the record where amount<210
deleted_rows = delta.delete(
    condition="amount < 210"
)

In [0]:
#Creating RAW (Unoptimized) Delta Table Using saveAsTable
import time
# Create RAW table (unoptimized)
start_write = time.time()
df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("chubb.default.orders_raw")
end_write = time.time()
# Print the elapsed time for the operation.
# This acts as a measurement before optimization
print("RAW table created as chubb.default.orders_raw")
print("Time taken to write RAW table:", end_write - start_write)


RAW table created as chubb.default.orders_raw
Time taken to write RAW table: 3.3099515438079834


In [0]:
#9.Use OPTIMIZE and optionally ZORDER on customer_id or order_date
spark.sql("""OPTIMIZE chubb.default.orders_raw""")
print("OPTIMIZE completed.")
# ZORDER on customer_id
spark.sql("""OPTIMIZE chubb.default.orders_raw ZORDER BY (customer_id)""")
print("ZORDER BY customer_id completed.")


OPTIMIZE completed.
ZORDER BY customer_id completed.


In [0]:
#10.show how small file problems can occur with too many partitions and how OPTIMIZE helps.
detail_df = spark.sql("DESCRIBE DETAIL chubb.default.orders_raw")
detail_df.select("numFiles", "sizeInBytes").show(truncate=False)

+--------+-----------+
|numFiles|sizeInBytes|
+--------+-----------+
|1       |269653     |
+--------+-----------+



In [0]:
import time
start = time.time()
spark.sql("SELECT COUNT(*) FROM chubb.default.orders_raw").collect()
end = time.time()

print("Time AFTER OPTIMIZE:", end - start)


Time AFTER OPTIMIZE: 0.45029187202453613
